Note: This a notebook in progress

In [3]:
import pandas as pd
PATH = "filtered.csv"
filtered = pd.read_csv(PATH)

In [278]:
filtered.head(5)

,Unnamed: 0,lemma,id_text,id_line,id_word,label
0,0,3(diš)[]NU,epsd2/admin/u3adm/P124149,3,P124149.3.1,o 1
1,1,udu[sheep]N,epsd2/admin/u3adm/P124149,3,P124149.3.2,o 1
2,2,6(diš)[]NU,epsd2/admin/u3adm/P124149,4,P124149.4.1,o 2
3,3,u[ewe]N,epsd2/admin/u3adm/P124149,4,P124149.4.2,o 2
4,4,2(diš)[]NU,epsd2/admin/u3adm/P124149,5,P124149.5.1,o 3


# Identifying People

Here, I create a dictionary with all the PN as keys. The values are dictionaries that contain profession and role mappings. In the future, I will add professions and roles to the PNs. The goal of this notebook is to finally have a csv with three columns for PN, associated role, and associated profession respectively. 

In [189]:
people = dict()
for row in filtered.itertuples():
    if 'PN' in row.lemma:
        people[(row.lemma, row.id_word)] = dict()

In [190]:
#sanity check
print(people.keys())

dict_keys([('{d}šul-gi-a-a-mu-ta[]PN', 'P405489.12.2'), ('{d}šul-gi-iri-mu[]PN', 'P405489.13.1'), ('{d}utu-ba-ni[]PN', 'P332036.11.1'), ('dingir-ki-ma-at[]PN', 'P332036.19.3'), ('di-ku₅-i₃-li₂[]PN', 'P332036.19.6'), ('la-gi-ip[]PN', 'P332036.23.3'), ('a₂-bi₂-lum[]PN', 'P332036.24.3'), ('dam-ku-um[]PN', 'P332036.25.3'), ('šu-ma-ma[]PN', 'P332036.26.3'), ('a-hu-we-er[]PN', 'P332036.47.1'), ('du-u₂-du[]PN', 'P332036.53.1'), ('na-lu₅[]PN', 'P332036.58.1'), ('ur-{d}ku₃-nun-na[]PN', 'P332036.64.1'), ('be-li₂-a-zu[]PN', 'P332036.68.1'), ('in-ta-e₃-a[]PN', 'P332036.70.1'), ('ur-{d}tilla₃[]PN', 'P320496.16.2'), ('maš-tur[]PN', 'P320496.17.1'), ('na-ap-la-nu-um[]PN', 'P320496.23.4'), ('ur-{d}nin-gal[]PN', 'P320496.24.2'), ('en-ša₃-ku₃-ge[]PN', 'P320496.29.2'), ('ARAD₂-mu[]PN', 'P320496.30.1'), ('bu-u₂-du[]PN', 'P320496.34.2'), ('maš-tur[]PN', 'P320496.35.1'), ('ur-{d}tilla₃[]PN', 'P320496.40.2'), ('maš-tur[]PN', 'P320496.41.1'), ('a-a-kal-la[]PN', 'P124307.7.1'), ('zu-ba-ga-ta[]PN', 'P124307.9.2

# Generate List of Professions Based on Location

In [15]:
#find professions <<INCOMPLETE as of 10.21.19>>

NOT_PROFESSIONS = {'dab[seize]V/t', 'maškim[administrator]N', 'šu[hand]N', 'ŋiri[foot]N', 'dumu[child]N', 'zig[rise]V/i', 'break', 'ki[place]N'}
i = 0
words = []
prev_row = None
for row in filtered.itertuples():
        if prev_row != None and 'PN' in prev_row.lemma and row.lemma not in NOT_PROFESSIONS:
            if 'PN' not in row.lemma and 'NU' not in row.lemma:
                words.append((row.lemma, row.id_word))
                i+=1
        prev_row = row
print(words)


[('nubanda[overseer]N', 'P332036.27.1'), ('itud[moon]N', 'P332036.71.1'), ('saŋŋa[official]N', 'P320496.16.3'), ('šagia[cup-bearer]N', 'P320496.17.2'), ('MAR.TU[westerner]N', 'P320496.23.5'), ('en[priest]N', 'P320496.29.3'), ('šagia[cup-bearer]N', 'P320496.35.2'), ('saŋŋa[official]N', 'P320496.40.3'), ('šagia[cup-bearer]N', 'P320496.41.2'), ('muhaldimgal[cook]N', 'P124307.22.1'), ('lugal[king]N', 'P128809.13.3'), ('lugal[king]N', 'P128769.11.3'), ('ezem[festival]N', 'P125988.21.1'), ('egia[bride]N', 'P125988.24.1'), ('dubsar[scribe]N', 'P125988.30.3'), ('dubsar[scribe]N', 'P125988.37.1'), ('itud[moon]N', 'P381729.14.1'), ('dim[create]V/t', 'P234930.25.4'), ('kurušda[fattener]N', 'P144583.15.3'), ('kuš[official]N', 'P107420.7.3'), ('kuš[official]N', 'P107420.21.3'), ('ugula[overseer]N', 'P107420.24.1'), ('kuš[official]N', 'P107420.31.3'), ('eg[levee]N', 'P117087.7.5'), ('dubsar[scribe]N', 'P212153.14.3'), ('dug[speak]V/t', 'P126670.5.1'), ('kurušda[fattener]N', 'P126670.14.3'), ('dubsar

# Finding Roles

More information on keywords can be found here:
https://github.com/niekveldhuis/Sumerian-network/blob/master/Finding%20Merging%20Roles%20and%20Professions/roles_and_keywords.md 

In [265]:
import re
def match_ki_ta():
    prev_row = None
    i=0
    words = []
    for row in filtered.itertuples():
        if re.match(r".+ta\[\]PN", row.lemma) and 'ki[place]N' in prev_row:
            words.append((row.lemma, row.id_word))
            i+=1
        prev_row = row
    print('Number of roles found:', i)
    return words


def find_keyword_prev(keyword):
    prev_row = None
    i=0
    words = []
    for row in filtered.itertuples():
        if prev_row != None and keyword in row.lemma and 'PN' in prev_row.lemma:
            words.append((prev_row.lemma, prev_row.id_word))
            i+=1
        prev_row = row
    print('Number of roles found:', i)
    return words
    
def find_keyword_next(keyword):
    prev_row = None
    i=0
    words = []
    for row in filtered.itertuples():
        if prev_row != None and keyword in prev_row.lemma and 'PN' in row.lemma:
            words.append((row.lemma, row.id_word))
            i+=1
        prev_row = row
    print('Number of roles found:', i)
    return words

1. Matching rows that have 'ki[place]N' in previous cell and '-ta' in current

In [266]:
#source
for person, id_w in match_ki_ta():
    try:
        people[(person, id_w)]['role'] = 'source'
    except:
        print(person, id_w)

Number of roles found: 9284


2. Matching rows that have 'dab[seize]V/t' in lemmatization

In [267]:
for (person, id_w) in find_keyword_prev('dab[seize]V/t'):
    try:
        people[(person, id_w)]['role'] = 'recipient'
    except:
        print(person, id_w)

Number of roles found: 5102


3. Matching rows that have 'mu.DU[delivery]N' in next cell

In [268]:
# new owner

for (person, id_w) in find_keyword_next('mu.DU[delivery]N'):
    try:
        people[(person, id_w)]['role'] = 'new owner'
    except:
        print((person, id_w))

Number of roles found: 2304


In [269]:
for (person, id_w) in find_keyword_prev('šu[hand]N'):
    try:
        people[(person, id_w)]['role'] = 'recipient'
    except:
        print(person, id_w)

Number of roles found: 2594


5. Matching rows that have 'ŋiri[foot]N' in next cell

In [270]:
for (person, id_w) in find_keyword_next('ŋiri[foot]N'):
    try:
        people[(person, id_w)]['role'] = 'intermediary'
    except:
        print((person, id_w))

Number of roles found: 3154


6. Looks for rows with 'maškim' in previous spot

In [271]:
for (person, id_w) in find_keyword_prev('maškim[administrator]N'):
    try:
        people[(person, id_w)]['role'] = 'representative'
    except:
        print((person, id_w))

Number of roles found: 2072


7. Looking for PNs before and after 'zig[rise]V/i'

In [272]:
for (person, id_w) in find_keyword_next('zig[rise]V/i'):
    try:
        people[(person, id_w)]['role'] = 'source'
    except:
        print((person, id_w))

Number of roles found: 12


In [273]:
for (person, id_w) in find_keyword_prev('zig[rise]V/i'):
    try:
        people[(person, id_w)]['role'] = 'source'
    except:
        print((person, id_w))

Number of roles found: 2383


This is every PN mentioned in the filtered texts

In [274]:
len(people.keys())

46126

In [275]:
people_df = pd.DataFrame(list(people.items()))
people_df.head(4)

,0,1
0,"({d}šul-gi-a-a-mu-ta[]PN, P405489.12.2)",{'role': 'source'}
1,"({d}šul-gi-iri-mu[]PN, P405489.13.1)",{'role': 'recipient'}
2,"({d}utu-ba-ni[]PN, P332036.11.1)",{}
3,"(dingir-ki-ma-at[]PN, P332036.19.3)",{}


# Transfer to CSV

In [280]:
name_word = pd.DataFrame(people_df[0].values.tolist(), index=people_df.index, columns = ['Name', 'id_word'])
final = name_word.join(people_df.drop(people_df.columns[0], axis = 1))
final.head(5)
final.to_csv('roles_professions.csv')

In [281]:
#TODO: need to deal with case when PN not right next to role
[(row.lemma, row.id_word) for row in filtered.itertuples() if 'P330639.17' in row.id_word]

[('a-hu-ni[]PN', 'P330639.17.1'),
 ('šagia[cup-bearer]N', 'P330639.17.2'),
 ('maškim[administrator]N', 'P330639.17.3')]